how this notebook works

In [ ]:
# dependencies

import pandas as pd

from ModelBuilderMethods import scaleColumns

setting up a dataset

splitting the dataset

balancing the dataset


normalization / scaling
0 MinMaxScaler           https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
1 MaxAbsScaler           https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html
2 StandardScaler         https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
3 RobustScaler           https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html
4 Normalizer             https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html
5 PowerTransformer       https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html
6 QuantileTransformer    https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html

categorical values one-hot encoding
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html


selecting a model

print model performance metrics on test data